In [1]:
import keras
from keras import ops
from keras import layers

In [19]:
class TransformerBlock(layers.Layer):
  def __init__(self, embed_dim, num_heads, ff_dim, rate = 0.1):
    super().__init__()
    self.att = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
    self.ffn = keras.Sequential([layers.Dense(ff_dim,activation = 'relu'), layers.Dense(embed_dim)])
    self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
    self.dropout1 = layers.Dropout(rate)
    self.dropout2 = layers.Dropout(rate)

  def call(self,input):
    attn_output = self.att(input,input)
    attn_output = self.dropout1(attn_output)
    out1 = self.layernorm1(input+attn_output)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output)
    return self.layernorm2(out1 + ffn_output)

In [15]:
class TokenAndPositionEmbedding(layers.Layer):
  def __init__(self,maxlen,vocab_size, embed_dim):
    super().__init__()
    self.token_emb = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)
    self.pos_emb = layers.Embedding(input_dim = maxlen, output_dim = embed_dim)
  def call(self,x):
    maxlen = ops.shape(x)[-1]
    position = ops.arange(start = 0, stop = maxlen,step = 1)
    position = self.pos_emb(position)
    x = self.token_emb(x)
    return x + position

In [11]:
vocab_size = 30000
maxlen = 200
(x_train,y_train), (x_val,y_val) = keras.datasets.imdb.load_data(num_words = vocab_size)

In [22]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [13]:
x_train = keras.utils.pad_sequences(x_train, maxlen = maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen = maxlen)

In [21]:
embed_dim = 32
num_heads = 2
ff_dim = 32

inputs = layers.Input(shape = (maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20,activation = 'relu')(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2,activation= 'softmax')(x)

model = keras.Model(inputs = inputs, outputs = outputs)

In [23]:
model.compile(optimizer = 'adam',loss= 'sparse_categorical_crossentropy',metrics = ['accuracy'] )
history = model.fit(x_train, y_train, batch_size = 32, epochs = 2, validation_data = (x_val, y_val))

Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 123s 149ms/step - accuracy: 0.6707 - loss: 0.5491 - val_accuracy: 0.8812 - val_loss: 0.2816
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 137s 143ms/step - accuracy: 0.9318 - loss: 0.1814 - val_accuracy: 0.8552 - val_loss: 0.3785
